<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Evaluation-of-metadata-predictions" data-toc-modified-id="Evaluation-of-metadata-predictions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Evaluation of metadata predictions</a></span></li><li><span><a href="#Set-up" data-toc-modified-id="Set-up-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set-up</a></span><ul class="toc-item"><li><span><a href="#Load-necessary-packages" data-toc-modified-id="Load-necessary-packages-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load necessary packages</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Load-embedding-model" data-toc-modified-id="Load-embedding-model-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Load embedding model</a></span></li><li><span><a href="#Create-dataframe" data-toc-modified-id="Create-dataframe-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Create dataframe</a></span></li></ul></li></ul></li><li><span><a href="#Prediction-performance" data-toc-modified-id="Prediction-performance-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Prediction performance</a></span><ul class="toc-item"><li><span><a href="#Covert-to-lower-case-for-better-comparison" data-toc-modified-id="Covert-to-lower-case-for-better-comparison-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Covert to lower case for better comparison</a></span></li><li><span><a href="#Overall-prediction-stats" data-toc-modified-id="Overall-prediction-stats-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Overall prediction stats</a></span></li><li><span><a href="#Exact-matches" data-toc-modified-id="Exact-matches-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Exact matches</a></span></li><li><span><a href="#Contained-within" data-toc-modified-id="Contained-within-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Contained within</a></span></li><li><span><a href="#Cosine-similarity" data-toc-modified-id="Cosine-similarity-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Cosine similarity</a></span></li><li><span><a href="#-Table-1--accuracy-of-prediction" data-toc-modified-id="-Table-1--accuracy-of-prediction-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span><font color="red"> Table 1 </font> accuracy of prediction</a></span><ul class="toc-item"><li><span><a href="#BLEU-scoring" data-toc-modified-id="BLEU-scoring-3.6.1"><span class="toc-item-num">3.6.1&nbsp;&nbsp;</span>BLEU scoring</a></span></li><li><span><a href="#Levenshtein-distance" data-toc-modified-id="Levenshtein-distance-3.6.2"><span class="toc-item-num">3.6.2&nbsp;&nbsp;</span>Levenshtein distance</a></span></li></ul></li></ul></li><li><span><a href="#Examples-of-misses?" data-toc-modified-id="Examples-of-misses?-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Examples of misses?</a></span></li></ul></div>

# Evaluation of metadata predictions
Adam Klie<br>
12/08/2019<br>
Script to evaluate predictions

# Set-up

## Load necessary packages

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import Levenshtein

from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

/cellar/users/aklie/opt/miniconda3/envs/PredictMEE_new/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/aklie/opt/miniconda3/envs/PredictMEE_new/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/cellar/users/aklie/opt/miniconda3/envs/PredictMEE_new/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/aklie/opt/miniconda3/envs/PredictMEE_new/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


## Load data

### Load embedding model

In [2]:
nlp = spacy.load('../data/wikipedia-pubmed-and-PMC-w2v')

/cellar/users/aklie/opt/miniconda3/envs/PredictMEE_new/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/aklie/opt/miniconda3/envs/PredictMEE_new/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


### Create dataframe

In [3]:
model_iter = '11_class'

In [4]:
grouping = pd.read_csv('../results/embedding/{model}/entity_merging.csv'.format(model=model_iter), index_col=0)

In [5]:
groups = grouping[grouping["I"] == 0]["GroupName"].values

In [6]:
import warnings
warnings.filterwarnings('ignore')
validation_data_count = {}
merged = pd.DataFrame()
total_count = 0
for group in groups:
    group_dir = group.replace(' ', '_')
    group_dir = group_dir.replace('/', '_')
    #if group == "Platform":
    #    continue
    predicted_df = pd.read_pickle(
    '../results/prediction/{mymodel}/{myclass}_prediction.pickle'.format(mymodel = model_iter, 
                                                                          myclass = group_dir))
    class_validation = pd.read_pickle(
    '../results/validation/{mymodel}/{myclass}_validation_values.pickle'.format(mymodel = model_iter, 
                                                                                myclass = group_dir))
    validation_data = pd.read_pickle(
    '../results/validation/{mymodel}/{myclass}_validation_set.pickle'.format(mymodel = model_iter, 
                                                                             myclass = group_dir))
    class_df = predicted_df[predicted_df['predicted'] == group]
    print(group, ":", validation_data.shape[0], ",", class_df.shape[0])
    validation_data_count[group] = validation_data.shape[0]
    tmp_df = class_validation.loc[class_df['srs'].values]
    tmp_df = tmp_df.loc[~tmp_df.index.duplicated(keep = 'first')]
    class_df['actual'] = tmp_df.loc[class_df['srs'].values]['value'].values
    merged = pd.concat([merged, class_df], axis=0)
    total_count = total_count + validation_data.shape[0]

Species : 1000 , 869
Strain : 1000 , 295
Cell type : 732 , 144
Genotype : 575 , 130
Condition/Disease : 126 , 23
Tissue : 1000 , 237
Sex : 184 , 4


KeyError: "None of [['SRS819363' 'SRS819365' 'ERS1026194' 'SRS716817']] are in the [index]"

# Prediction performance

## Covert to lower case for better comparison

In [ ]:
merged["token"] = merged["token"].str.lower()
merged["actual"] = merged["actual"].str.lower()

## Overall prediction stats

In [ ]:
predicted_count = merged.shape[0]
print(('%d/%d (%.2f%%) predicted' % 
       (predicted_count, total_count, predicted_count/total_count*100)))

In [ ]:
merged["predicted"].value_counts()

In [ ]:
# missing_df = class_validation[~class_validation.index.isin(class_df['srs'].values)]
# missing = validation_data[~validation_data.index.get_level_values(0).isin(class_df['srs'].values)]
# if missing_df.shape[0] == len(missing):  # if there are not duplicate srs predictions
#     missing_df['TITLE'] = missing.values

In [ ]:
# missing_df.to_csv(
#     '../results/{model}/predictions/{myclass}_missing.csv'.format(model = model_iter, myclass = predicted_class))

## Exact matches

In [ ]:
merged["exact_match"] = merged['token'] == merged['actual']

In [ ]:
num_correct = merged["exact_match"].sum()
print(('%d/%d (%.2f%%) exact matches' % 
       (num_correct, predicted_count, num_correct/predicted_count*100)))

In [ ]:
class_exact = merged.groupby(['predicted', 'exact_match']).size().reset_index().pivot(index='predicted', 
                                                                                      columns='exact_match')
class_exact

In [ ]:
merged.groupby('exact_match').mean()["score"]

In [ ]:
sns.distplot(merged[merged['exact_match'] == True]['score'], label='Match', kde=False)
sns.distplot(merged[merged['exact_match'] == False]['score'], label='Non-match', kde=False)
plt.legend(prop={'size': 12})

In [ ]:
# exact_correct_df.to_csv(
#     '../results/{model}/predictions/{myclass}_correct.csv'.format(model = model_iter, myclass = predicted_class))
# exact_incorrect_df.to_csv(
#     '../results/{model}/predictions/{myclass}_incorrect.csv'.format(model = model_iter, myclass = predicted_class))

In [ ]:
ax = sns.violinplot(x='predicted', y='score', data=merged, hue='exact_match')

## Contained within

In [ ]:
def within(x):
    if (x["actual"] in x['token']) or (x['token'] in x["actual"]):
        return True
    else:
        return False

In [ ]:
merged['within'] = merged.apply(within, axis=1)
count = merged['within'].sum()

In [ ]:
print(('%d/%d (%.2f%%) of predicted contained in actual' % 
       (count, predicted_count, count/predicted_count*100)))

In [ ]:
class_contain = merged.groupby(['predicted', 'within']).size().reset_index().pivot(index='predicted', 
                                                                                      columns='within')
class_contain.columns = ["non-match", "match"]
class_contain["fraction_match"] = class_contain["match"]/class_contain.sum(axis=1)

In [ ]:
class_contain

## Cosine similarity

In [ ]:
predicted_docs = nlp.pipe(merged['token'].tolist())
predicted_vectors = np.array([doc.vector for doc in predicted_docs])

actual_docs = nlp.pipe(merged['actual'].tolist())
actual_vectors = np.array([doc.vector for doc in actual_docs])

In [ ]:
similarities = []
for i in range(len(predicted_vectors)):
    cosine_sim = cosine_similarity(predicted_vectors[i].reshape(1,200), actual_vectors[i].reshape(1,200))
    similarities.append(float(cosine_sim))

In [ ]:
merged["cosine_similarity"] = similarities

In [ ]:
plt.hist(merged['cosine_similarity'])

In [ ]:
sim_threshold = 0.70
merged["cosine_match"] = merged['cosine_similarity'].values > sim_threshold
num_sim = merged["cosine_match"].sum()

In [ ]:
print(('%d/%d (%.2f%%) Predicted with cosine similarity greater than %.1f\n' 
                     % (num_sim, predicted_count, num_sim/predicted_count*100, sim_threshold)))

In [ ]:
class_cosine = merged.groupby(['predicted', 'cosine_match']).size().reset_index().pivot(index='predicted', 
                                                                                        columns='cosine_match')
class_cosine.columns = ["non-match", "match"]
class_cosine["fraction_match"] = class_cosine["match"]/class_cosine.sum(axis=1)

In [ ]:
class_cosine

In [ ]:
title_counts = pd.DataFrame(pd.Series(validation_data_count), columns=['titles'])

## <font color='red'> Table 1 </font> accuracy of prediction

In [ ]:
merged["either"] = merged["within"] | merged["cosine_match"]

In [ ]:
class_either = merged.groupby(['predicted', 'either']).size().reset_index().pivot(index='predicted', 
                                                                                        columns='either')
class_either.columns = ["non-match", "match"]
class_either["either_match"] = class_either["match"]/class_either.sum(axis=1)

In [ ]:
class_either

In [ ]:
table1 = class_cosine.merge(pd.DataFrame(merged["predicted"].value_counts()), left_index=True, right_index=True)
table1 = table1.merge(title_counts, left_index=True, right_index=True)
table1 = table1.merge(class_contain, left_index=True, right_index=True, suffixes=['_cosine', '_contain'])
table1 = table1.merge(class_either, left_index=True, right_index=True, suffixes=['_either', '_either'])

In [ ]:
table1["fraction_predicted"] = table1['predicted']/table1['titles']

In [ ]:
table1 = table1[['titles', 'predicted', 'fraction_predicted', 
        'match_cosine', 'fraction_match_cosine', 'match_contain', 'fraction_match_contain', 'match', 'either_match']]

In [ ]:
table1['percent_correct'] = table1[["fraction_match_cosine", "fraction_match_contain"]].max(axis=1)

In [ ]:
table1.head()

In [ ]:
table1.to_csv('../doc/tables/Table1.csv', index=True)

### BLEU scoring

In [ ]:
def bleu1(x):
    return sentence_bleu(x["predicted"], x["actual"], weights=(1, 0, 0, 0), smoothing_function=smoother.method1)

def bleu4(x):
    return sentence_bleu(x["predicted"], x["actual"], weights=(0, 0, 0, 1), smoothing_function=smoother.method1)

In [ ]:
smoother = SmoothingFunction()
table1["BLUE1"] = table1.apply(bleu1, axis=1)
table1["BLUE4"] = table1.apply(bleu4, axis=1)

### Levenshtein distance

In [ ]:
def levenshtein(x):
    return Levenshtein.distance(x["predicted"], x["actual"])

In [ ]:
table1["levenshtein"] = table1.apply(levenshtein, axis=1)

# Examples of misses?

In [ ]:
missed = merged[~merged["either"]]
missed_df = missed[["predicted", "orig_text","recovered_txt", "actual"]]

In [ ]:
missed_df.to_csv('../doc/figures/Supplementary/Supp_Table3.csv')

In [ ]:
missed.columns